In [1]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [2]:
import google.generativeai as genai
import os

genai.configure(api_key=GEMINI_API_KEY)

model = genai.GenerativeModel(model_name="gemini-1.5-flash")
response = model.generate_content("Explain how AI works")
print(response.text)


## Understanding the Inner Workings of AI

Artificial Intelligence (AI) is a broad field that encompasses various techniques for building intelligent machines. It's not a single thing, but rather a collection of approaches, tools, and algorithms designed to mimic human intelligence.

**Here's a simplified breakdown of how AI works:**

**1. Data is King:** AI systems are trained on massive datasets, which act as the learning material. These datasets can contain images, text, sound, or other data depending on the task.

**2. Algorithms: The Brains of the Operation:** AI systems use different algorithms to analyze the data and learn patterns. Some common algorithms include:

* **Machine Learning:** These algorithms learn from data and make predictions or decisions without explicit programming.
* **Deep Learning:** A subset of machine learning that uses artificial neural networks with multiple layers to learn complex patterns.
* **Natural Language Processing (NLP):** Enables computers to u

In [3]:
import json
import time

In [10]:
def make_api_call(messages, max_tokens, is_final_answer=False, custom_model=None):
    model = custom_model or genai.GenerativeModel('gemini-pro')

    for attempt in range(3):
        try:
            chat = model.start_chat(history=[])

            # Combine all messages into a single prompt
            combined_prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])

            response = chat.send_message(combined_prompt)

            if is_final_answer:
                return response.text
            else:
                return json.loads(response.text)
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error", "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}", "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying

In [11]:
def generate_response(prompt, custom_model=None):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant that explains your reasoning step by step. For each step, provide a title that describes what you're doing in that step, along with the content. Decide if you need another step or if you're ready to give the final answer. Respond in JSON format with 'title', 'content', and 'next_action' (either 'continue' or 'final_answer') keys. USE AS MANY REASONING STEPS AS POSSIBLE. AT LEAST 3. BE AWARE OF YOUR LIMITATIONS AS AN LLM AND WHAT YOU CAN AND CANNOT DO. IN YOUR REASONING, INCLUDE EXPLORATION OF ALTERNATIVE ANSWERS. CONSIDER YOU MAY BE WRONG, AND IF YOU ARE WRONG IN YOUR REASONING, WHERE IT WOULD BE. FULLY TEST ALL OTHER POSSIBILITIES. YOU CAN BE WRONG. WHEN YOU SAY YOU ARE RE-EXAMINING, ACTUALLY RE-EXAMINE, AND USE ANOTHER APPROACH TO DO SO. DO NOT JUST SAY YOU ARE RE-EXAMINING. USE AT LEAST 3 METHODS TO DERIVE THE ANSWER. USE BEST PRACTICES.

Example of a valid JSON response:
```json
{
    "title": "Identifying Key Information",
    "content": "To begin solving this problem, we need to carefully examine the given information and identify the crucial elements that will guide our solution process. This involves...",
    "next_action": "continue"
}```
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "Thank you! I will now think step by step following my instructions, starting at the beginning after decomposing the problem."}
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 300, custom_model=custom_model)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        messages.append({"role": "assistant", "content": json.dumps(step_data)})

        if step_data['next_action'] == 'final_answer' or step_count > 25: # Maximum of 25 steps to prevent infinite thinking time. Can be adjusted.
            break

        step_count += 1

        # Yield after each step for Streamlit to update
        yield steps, None  # We're not yielding the total time until the end

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based solely on your reasoning above. Do not use JSON formatting. Only provide the text response without any titles or preambles. Retain any formatting as instructed by the original prompt, such as exact formatting for free response or multiple choice."})

    start_time = time.time()
    final_data = make_api_call(messages, 1200, is_final_answer=True, custom_model=custom_model)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    steps.append(("Final Answer", final_data, thinking_time))

    yield steps, total_thinking_time

In [12]:
user_input = 'how many r\'s are in strawberry'
steps_generator = generate_response(user_input)

for steps, total_time in steps_generator:
  # Clear the console (works for Unix-like systems)
  #os.system('clear')

  print(f"Prompt: {user_input}\n")
  for step_title, step_content, step_time in steps:
      print(f"{step_title} (Time: {step_time:.2f}s)")
      if isinstance(step_content, str):
          print(step_content)
      elif isinstance(step_content, dict):
          print(json.dumps(step_content, indent=2))
      else:
          print(f"Unexpected content type: {type(step_content)}")
      print()

Prompt: how many r's are in strawberry

Step 1: Error (Time: 9.59s)
Failed to generate step after 3 attempts. Error: Expecting value: line 1 column 1 (char 0)

Final Answer (Time: 1.96s)
I am sorry, I am unable to provide a final answer based solely on my reasoning above. The error message indicates that I have failed to generate a valid step after 3 attempts, and I cannot continue with the reasoning process.



In [15]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [24]:
import openai
import os
import json
import time

client = openai.OpenAI(api_key = userdata.get('OPEN_AI_KEY') )

In [18]:
def make_api_call(messages, max_tokens, is_final_answer=False):
    for attempt in range(3):
        try:
            response = client.chat.completions.create(
                model="gpt-4o",  # Using GPT-3.5 Turbo, adjust as needed
                messages=messages,
                max_tokens=max_tokens,
                temperature=0.2,
                response_format={"type": "json_object"}
            )
            return json.loads(response.choices[0].message.content)
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error",
                            "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}",
                            "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying


In [19]:
def generate_response(prompt):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant with advanced reasoning capabilities. Your task is to provide detailed, step-by-step explanations of your thought process. For each step:

1. Provide a clear, concise title describing the current reasoning phase.
2. Elaborate on your thought process in the content section.
3. Decide whether to continue reasoning or provide a final answer.

Response Format:
Use JSON with keys: 'title', 'content', 'next_action' (values: 'continue' or 'final_answer')

Key Instructions:
- Employ at least 5 distinct reasoning steps.
- Acknowledge your limitations as an AI and explicitly state what you can and cannot do.
- Actively explore and evaluate alternative answers or approaches.
- Critically assess your own reasoning; identify potential flaws or biases.
- When re-examining, employ a fundamentally different approach or perspective.
- Utilize at least 3 diverse methods to derive or verify your answer.
- Incorporate relevant domain knowledge and best practices in your reasoning.
- Quantify certainty levels for each step and the final conclusion when applicable.
- Consider potential edge cases or exceptions to your reasoning.
- Provide clear justifications for eliminating alternative hypotheses.


Example of a valid JSON response:
```json
{
    "title": "Initial Problem Analysis",
    "content": "To approach this problem effectively, I'll first break down the given information into key components. This involves identifying...[detailed explanation]... By structuring the problem this way, we can systematically address each aspect.",
    "next_action": "continue"
}```
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant",
         "content": "Thank you! I will now think step by step following my instructions, starting at the beginning after decomposing the problem."}
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 300)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        messages.append({"role": "assistant", "content": json.dumps(step_data)})

        if step_data['next_action'] == 'final_answer':
            break

        step_count += 1

        # Yield after each step for Streamlit to update
        yield steps, None  # We're not yielding the total time until the end

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based on your reasoning above."})

    start_time = time.time()
    final_data = make_api_call(messages, 200, is_final_answer=True)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    steps.append(("Final Answer", final_data['content'], thinking_time))

    yield steps, total_thinking_time

In [25]:
def make_api_call(messages, max_tokens, is_final_answer=False):
    for attempt in range(3):
        try:
            response = client.chat.completions.create(
                model="gpt-4",  # Using GPT-4, adjust as needed
                messages=messages,
                max_tokens=max_tokens,
                temperature=0.2
            )
            return json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
            # If JSON parsing fails, return the raw content
            return {"title": "Parsing Error", "content": response.choices[0].message.content, "next_action": "final_answer"}
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error",
                            "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}",
                            "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying

def generate_response(prompt):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant with advanced reasoning capabilities. Your task is to provide detailed, step-by-step explanations of your thought process. For each step:

1. Provide a clear, concise title describing the current reasoning phase.
2. Elaborate on your thought process in the content section.
3. Decide whether to continue reasoning or provide a final answer.

Response Format:
Use JSON with keys: 'title', 'content', 'next_action' (values: 'continue' or 'final_answer')

Key Instructions:
- Employ at least 5 distinct reasoning steps.
- Acknowledge your limitations as an AI and explicitly state what you can and cannot do.
- Actively explore and evaluate alternative answers or approaches.
- Critically assess your own reasoning; identify potential flaws or biases.
- When re-examining, employ a fundamentally different approach or perspective.
- Utilize at least 3 diverse methods to derive or verify your answer.
- Incorporate relevant domain knowledge and best practices in your reasoning.
- Quantify certainty levels for each step and the final conclusion when applicable.
- Consider potential edge cases or exceptions to your reasoning.
- Provide clear justifications for eliminating alternative hypotheses.
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant",
         "content": "Understood. I will now think through this step-by-step, following the given instructions and starting by decomposing the problem."}
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 300)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        messages.append({"role": "assistant", "content": json.dumps(step_data)})

        if step_data['next_action'] == 'final_answer':
            break

        step_count += 1

        # Print each step
        print(f"\nStep {step_count}: {step_data['title']}")
        print(step_data['content'])
        print(f"Thinking time: {thinking_time:.2f} seconds")

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based on your reasoning above."})

    start_time = time.time()
    final_data = make_api_call(messages, 200, is_final_answer=True)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    steps.append(("Final Answer", final_data['content'], thinking_time))

    # Print final answer
    print("\nFinal Answer:")
    print(final_data['content'])
    print(f"Thinking time: {thinking_time:.2f} seconds")

    print(f"\nTotal thinking time: {total_thinking_time:.2f} seconds")

    return steps, total_thinking_time

print("OpenAI Reasoning Chains")
print("This is a prototype using OpenAI's GPT model to create reasoning chains for improved output accuracy.")
print("The accuracy has not been formally evaluated yet.")

while True:
    user_query = input("\nEnter your query (or 'quit' to exit): ")

    if user_query.lower() == 'quit':
        break

    print("\nGenerating response...")

    generate_response(user_query)



OpenAI Reasoning Chains
This is a prototype using OpenAI's GPT model to create reasoning chains for improved output accuracy.
The accuracy has not been formally evaluated yet.

Enter your query (or 'quit' to exit): how many r's in strawberry

Generating response...

Step 2: Problem Decomposition
The task is to count the number of occurrences of the letter 'r' in the word 'strawberry'. This is a straightforward task that involves scanning each character in the word and incrementing a counter each time the character 'r' is encountered.
Thinking time: 3.58 seconds

Step 3: Algorithm Design
I will use a simple algorithm to solve this problem. The algorithm will iterate over each character in the word 'strawberry'. For each character, it will check if the character is 'r'. If it is, it will increment a counter. At the end of the algorithm, the counter will hold the total number of 'r's in the word 'strawberry'.
Thinking time: 3.29 seconds

Step 4: Algorithm Execution
Now I will execute the 

KeyboardInterrupt: 

In [26]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [28]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00


In [31]:
import os
import json
import time
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.callbacks import get_openai_callback

def make_api_call(messages, max_tokens, is_final_answer=False):
    llm = ChatOpenAI(model_name="gpt-4", temperature=0.2, max_tokens=max_tokens,openai_api_key=userdata.get('OPEN_AI_KEY'))

    for attempt in range(3):
        try:
            with get_openai_callback() as cb:
                response = llm(messages)

            try:
                return json.loads(response.content)
            except json.JSONDecodeError:
                # If JSON parsing fails, return the raw content
                return {"title": "Parsing Error", "content": response.content, "next_action": "final_answer"}

        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error",
                            "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}",
                            "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying

def generate_response(prompt):
    system_message = SystemMessage(content="""You are an expert AI assistant with advanced reasoning capabilities. Your task is to provide detailed, step-by-step explanations of your thought process. For each step:

1. Provide a clear, concise title describing the current reasoning phase.
2. Elaborate on your thought process in the content section.
3. Decide whether to continue reasoning or provide a final answer.

Response Format:
Use JSON with keys: 'title', 'content', 'next_action' (values: 'continue' or 'final_answer')

Key Instructions:
- Employ at least 5 distinct reasoning steps.
- Acknowledge your limitations as an AI and explicitly state what you can and cannot do.
- Actively explore and evaluate alternative answers or approaches.
- Critically assess your own reasoning; identify potential flaws or biases.
- When re-examining, employ a fundamentally different approach or perspective.
- Utilize at least 3 diverse methods to derive or verify your answer.
- Incorporate relevant domain knowledge and best practices in your reasoning.
- Quantify certainty levels for each step and the final conclusion when applicable.
- Consider potential edge cases or exceptions to your reasoning.
- Provide clear justifications for eliminating alternative hypotheses.
""")

    messages = [
        system_message,
        HumanMessage(content=prompt),
        AIMessage(content="Understood. I will now think through this step-by-step, following the given instructions and starting by decomposing the problem.")
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 300)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        messages.append(AIMessage(content=json.dumps(step_data)))

        if step_data['next_action'] == 'final_answer':
            break

        step_count += 1

        # Print each step
        print(f"\nStep {step_count}: {step_data['title']}")
        print(step_data['content'])
        print(f"Thinking time: {thinking_time:.2f} seconds")

    # Generate final answer
    messages.append(HumanMessage(content="Please provide the final answer based on your reasoning above."))

    start_time = time.time()
    final_data = make_api_call(messages, 200, is_final_answer=True)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    steps.append(("Final Answer", final_data['content'], thinking_time))

    # Print final answer
    print("\nFinal Answer:")
    print(final_data['content'])
    print(f"Thinking time: {thinking_time:.2f} seconds")

    print(f"\nTotal thinking time: {total_thinking_time:.2f} seconds")

    return steps, total_thinking_time

print("LangChain Reasoning Chains")
print("This is a prototype using LangChain with OpenAI's GPT model to create reasoning chains for improved output accuracy.")
print("The accuracy has not been formally evaluated yet.")

while True:
    user_query = input("\nEnter your query (or 'quit' to exit): ")

    if user_query.lower() == 'quit':
        break

    print("\nGenerating response...")

    generate_response(user_query)


LangChain Reasoning Chains
This is a prototype using LangChain with OpenAI's GPT model to create reasoning chains for improved output accuracy.
The accuracy has not been formally evaluated yet.

Enter your query (or 'quit' to exit): how many r's in strawberry

Generating response...


<ipython-input-31-2e835c832b57>:14: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)



Step 2: Problem Decomposition
The task is to determine the number of occurrences of the letter 'r' in the word 'strawberry'. This involves breaking down the word into individual letters and counting the number of times 'r' appears.
Thinking time: 3.83 seconds

Step 3: Direct Counting
The word 'strawberry' is composed of the following letters: 's', 't', 'r', 'a', 'w', 'b', 'e', 'r', 'r', 'y'. By directly counting, we can see that the letter 'r' appears 3 times.
Thinking time: 4.00 seconds

Step 4: Alternative Approach: String Analysis
An alternative approach to counting the occurrences of a letter in a word is to use string analysis methods. For example, in programming, one could use a function to count the number of 'r's in 'strawberry'. However, as an AI, I am not currently executing code, but rather explaining my thought process. In this case, the direct counting method is more appropriate.
Thinking time: 5.26 seconds

Step 5: Assessing Potential Errors
In this task, potential error

In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import openai
import json
import os

# Set up OpenAI API Key


# Initialize the OpenAI client using LangChain's wrapper
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.2,api_key=userdata.get('OPEN_AI_KEY'))

# Function to generate customized exam MCQs step-by-step using LangChain
def generate_mcq(prompt):
    # Define the initial system message with detailed step-by-step instructions
    system_prompt = """
You are an expert AI assistant with advanced reasoning capabilities. Your task is to create high-quality multiple-choice questions (MCQs) for exams. Follow a structured, step-by-step approach using the template below:

### Steps to Follow:
1. **Topic Analysis and Objective Definition**:
   - Title: "Identifying Key Concepts"
   - Content: Analyze the provided topic(s) and grade level. Identify the key concepts, skills, or learning objectives that each question should target.
   - Next Action: `continue`

2. **Question Type and Format Selection**:
   - Title: "Defining MCQ Structure"
   - Content: Choose the type of MCQ (e.g., conceptual, application-based, factual). Define the question format, ensuring it aligns with the learning objective. Select an appropriate level of difficulty (easy, medium, hard) for each question.
   - Next Action: `continue`

3. **Drafting the MCQ Statement**:
   - Title: "Creating the Question Stem"
   - Content: Draft the main statement (stem) of the MCQ, ensuring clarity, conciseness, and alignment with the learning objective. Ensure there is no ambiguity or double negatives.
   - Next Action: `continue`

4. **Designing the Answer Options**:
   - Title: "Generating Options and Distractors"
   - Content: Create 4 distinct options (A, B, C, D), including the correct answer and 3 plausible distractors. Ensure that the distractors are carefully chosen to test conceptual understanding and prevent guessing.
   - Next Action: `continue`

5. **Validating the MCQ**:
   - Title: "Reviewing and Refining"
   - Content: Review the entire question, check for any biases, misleading wording, or unintended clues. Ensure the correct answer is logically derived and unambiguous.
   - Next Action: `continue`

6. **Assigning Metadata**:
   - Title: "Adding Metadata and Labels"
   - Content: Label the question with metadata: topic, subtopic, grade level, cognitive level (e.g., Bloom's Taxonomy), and expected solution time.
   - Next Action: `continue`

7. **Final Quality Check and Output**:
   - Title: "Quality Assurance and Final Output"
   - Content: Conduct a final review and output the formatted question with metadata. Structure the output in JSON with: `question`, `options` (list), `correct_answer`, `explanation`, and `metadata`.
   - Next Action: `final_answer`
"""

    # Start the conversation with the system prompt
    messages = [SystemMessage(content=system_prompt), HumanMessage(content=prompt)]

    steps = []
    step_count = 1

    # Run through the step-by-step process until `final_answer` is reached
    while True:
        response = chat_model(messages)

        # Parse the content of the AI response
        response_content = response.content
        try:
            step_data = json.loads(response_content)
        except json.JSONDecodeError:
            print(f"Error decoding JSON at step {step_count}: {response_content}")
            break

        # Append step to the reasoning chain
        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content']))

        # Print out each step for clarity
        print(f"Step {step_count}: {step_data['title']}\n{step_data['content']}\n{'=' * 50}")

        # Check if final answer has been reached
        if step_data['next_action'] == 'final_answer':
            break

        # Add AI response back to the conversation for the next step
        messages.append(AIMessage(content=response_content))

        # Continue with the next step
        messages.append(HumanMessage(content="Continue to the next step."))

        step_count += 1

    # Final output
    #print("\nFinal MCQ Generated:")
    #print(steps)  # Print the content of the final step
    return steps

# Example usage

# Example prompt for generating 5 Algebra MCQs for Grade 9
sample_prompt = "Create 5 multiple-choice questions for Grade 9 on Algebra covering topics such as linear equations and factorization."

# Generate the MCQs using LangChain and OpenAI
generate_mcq(sample_prompt)


Error decoding JSON at step 1: ### Steps to Follow:

1. **Topic Analysis and Objective Definition**:
   - Title: "Identifying Key Concepts"
   - Content: The topics are linear equations and factorization for Grade 9 Algebra. Key concepts include solving linear equations, understanding the properties of equality, and applying factorization techniques to simplify expressions and solve quadratic equations.
   - Next Action: `continue`

2. **Question Type and Format Selection**:
   - Title: "Defining MCQ Structure"
   - Content: The questions will be a mix of conceptual and application-based types. The difficulty level will range from easy to medium to ensure a comprehensive assessment of students' understanding.
   - Next Action: `continue`

3. **Drafting the MCQ Statement**:
   - Title: "Creating the Question Stem"
   - Content: Drafting the stems for each question, ensuring they are clear and aligned with the learning objectives.
   - Next Action: `continue`

4. **Designing the Answer O

[]

In [37]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import json

# Set up your OpenAI API key
#openai.api_key = "your-openai-api-key"  # Replace with your actual OpenAI API key

# Initialize the OpenAI client using LangChain's wrapper
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.2,api_key=userdata.get('OPEN_AI_KEY'))

# Check the API key to ensure it's configured correctly
#assert openai.api_key, "Please set up your OpenAI API key in the previous cell."


In [68]:
# Function to generate and modify MCQs step-by-step using LangChain
def generate_mcq_step_by_step(prompt):
    # Initial system message and instructions for iterative generation
    system_prompt = """
You are an advanced AI assistant with the task of creating high-quality multiple-choice questions (MCQs) for Grade 9 exams. Your goal is to iteratively build the questions step-by-step until they are fully complete. Follow these steps:

1. Generate initial questions:
   - Create 5 basic question stems.
   - Explore alternative question formulations.

2. Add answer options:
   - Provide 4 options (A, B, C, D) for each question.
   - Use diverse methods to derive and verify answers.
   - Critically assess for potential biases or flaws.

3. Review and improve:
   - Refine for clarity and precision.
   - Consider edge cases and exceptions.
   - Re-examine using a different perspective.

4. **Add Metadata**:
   - Add metadata to each question, including:
     - Topic
     - Subtopic
     - Cognitive level
     - Expected solution time

5. **Compile Final Questions**:
   - Compile all completed questions into the final output format with structured JSON.



### Output Format:
Ensure each response is in valid JSON format with real data. Use the following structure for each output:

{
  "questions": [
    {
      "question": "<question stem>",
      "options": ["<option A>", "<option B>", "<option C>", "<option D>"],
      "correct_answer": "<correct option>",
      "explanation": "<explanation>",
      "metadata": {
        "topic": "<topic>",
        "subtopic": "<subtopic>",
        "grade_level": "<grade level>",
        "cognitive_level": "<cognitive level>",
        "expected_solution_time": "<time>"
      }
    }
  ]
}

"""

    # Start the conversation with the system prompt
    messages = [SystemMessage(content=system_prompt), HumanMessage(content=prompt)]

    # Store the intermediate results in each step
    current_questions = []

    # Steps for the iterative process
    steps = [
        "Generate the basic question stems for the MCQs.",
        "Add answer options and distractors to each question.",
        "Review the questions and options for clarity and accuracy.",
        "Add metadata to each question.",
        "Compile the final structured MCQs."
    ]

    # Process each step one by one
    for step_index, step_instruction in enumerate(steps):
        # Send current questions and instructions for the next modification step
        if step_index == 0:
            # For the first step, we just use the initial prompt
            print(f"Step {step_index + 1}: {step_instruction}")
        else:
            # Include the current question structure for subsequent steps
            questions_message = f"Here is the current state of the questions:\n{json.dumps({'questions': current_questions}, indent=4)}"
            messages.append(HumanMessage(content=questions_message))
            print(f"\nStep {step_index + 1}: {step_instruction}\n")

        # Instruct the assistant to modify based on the current step
        messages.append(HumanMessage(content=step_instruction))

        # Get response from the assistant
        response = chat_model(messages)

        # Parse and display raw response for debugging
        response_content = response.content.strip()
        print(f"Raw response at step {step_index + 1}:\n{response_content}\n{'=' * 80}")

        # Convert the response to JSON
        try:
            step_data = json.loads(response_content)
            if "questions" in step_data:
                current_questions = step_data["questions"]
            else:
                print(f"Step {step_index + 1} did not produce valid questions JSON.")
        except json.JSONDecodeError:
            print(f"Error decoding JSON at step {step_index + 1}: {response_content}")
            break

        # Add the assistant's response to the message history
        messages.append(AIMessage(content=response_content))

    # Final Output
    print("\nFinal MCQs Generated:\n")
    if current_questions:
        final_output = {"questions": current_questions}
        print(json.dumps(final_output, indent=4))
    else:
        print("No questions were generated.")

    return current_questions


In [69]:
# Example prompt for generating 5 Algebra MCQs for Grade 9
sample_prompt = "Create 5 multiple-choice questions for Grade 9 on Algebra, covering topics such as linear equations and factorization."

# Generate the MCQs using the step-by-step approach
generated_mcqs = generate_mcq_step_by_step(sample_prompt)

# Display the final MCQ generated in a readable format
if generated_mcqs:
    print("\nGenerated Questions:\n")
    for question in generated_mcqs:
        print(f"Question: {question['question']}")
        print(f"Options: {question['options']}")
        print(f"Correct Answer: {question['correct_answer']}")
        print(f"Explanation: {question['explanation']}")
        print(f"Metadata: {question['metadata']}\n")
else:
    print("No questions were generated.")


Step 1: Generate the basic question stems for the MCQs.
Raw response at step 1:
{
  "questions": [
    {
      "question": "What is the solution to the linear equation 2x + 3 = 11?",
      "options": [],
      "correct_answer": "",
      "explanation": "",
      "metadata": {}
    },
    {
      "question": "Which of the following expressions is a factor of x^2 - 5x + 6?",
      "options": [],
      "correct_answer": "",
      "explanation": "",
      "metadata": {}
    },
    {
      "question": "If y = 3x + 2, what is the value of y when x = 4?",
      "options": [],
      "correct_answer": "",
      "explanation": "",
      "metadata": {}
    },
    {
      "question": "What is the slope of the line represented by the equation y = -2x + 5?",
      "options": [],
      "correct_answer": "",
      "explanation": "",
      "metadata": {}
    },
    {
      "question": "Which of the following is the factored form of the quadratic equation x^2 + 7x + 10?",
      "options": [],
      "cor

In [72]:
!pip install langchain pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 4.5 MB/s eta 0:00:00


In [81]:
# Cell 1: Install necessary libraries
!pip install langchain pypdf pdfminer.six


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.4 MB/s eta 0:00:00


In [104]:
# Cell 1: Install Required Libraries
!pip install PyPDF2 langchain openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.4 MB/s eta 0:00:00


In [106]:
# Cell 2: Import Libraries and Setup
import PyPDF2
import re
import json
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Set your OpenAI API key (make sure to set this in Colab or securely via Google Colab secrets)
openai_api_key = userdata.get('OPEN_AI_KEY') # Replace with your actual OpenAI API key

# Initialize the chat model with the API key
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=openai_api_key)


In [186]:
# Cell 2: Extract and Display Full Text from PDF
import PyPDF2

def extract_text_from_pdf(pdf_path):
    """Extracts and returns the full text from a PDF file."""
    try:
        # Read the PDF file
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            full_text = ""
            for page in reader.pages:
                full_text += page.extract_text() + "\n"
        return full_text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""

# Specify your PDF path (make sure to upload your PDF to Colab)
pdf_path = "/content/differential-matter_e.pdf"  # Change this to the name of your PDF file in Colab

# Extract and display the full text
full_text = extract_text_from_pdf(pdf_path)
print("Full Text Extracted from PDF:\n")
print(full_text)


Full Text Extracted from PDF:

1Mathematics study guide on the Egyptian Knowledge Bank (EKB) - secondary stage
 Subject: Calculs                          Grade: third secondary                           2021/2022
Lesson Learning Outcomes The digital resources available on EKB
Student Book Najwa Limited
Differentiation and its Applications
Differentiation of trigo-
nometric
 function•	 To find derivatives of the inverse of trigonometric functions 
(secant – cosecant - cotan)https://d3sk34bfh9epsl.
cloudfront.net/mathematics/differential-integral-calculus/g12/english/unit1_lesson1.pdfhttps://lms.ekb.eg/repository/resource/80cb0859-7381-404b-998f-14be59bf7f26/en 
Implicit and 
parametric
 differentiation•	 To find derivatives of (explicit, implicit, parametric …..) 
functionshttps://d3sk34bfh9epsl.
cloudfront.net/mathematics/differential-integral-calculus/g12/english/unit1_lesson2.pdfhttps://lms.ekb.eg/repository/resource/f9cc1bbf-a102-45a3-8583-9f5086a1d4df
•	 To solve problems on deriva

In [125]:
# Cell 3: Process the Full Text Using OpenAI
import json
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

# Set up OpenAI API key (Make sure to securely input your API key in Colab)
openai_api_key = userdata.get('OPEN_AI_KEY')  # Replace with your actual OpenAI API key

# Initialize the OpenAI model
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=openai_api_key)

def process_text_with_openai(full_text, chat_model):
    """Processes the extracted text to identify and format MCQs using OpenAI."""
    # Define the system prompt
    system_prompt = """
    You are an advanced AI assistant that extracts and processes multiple-choice questions (MCQs) from text.
    Please extract the following for each identified question:

    1. The question stem.
    2. Four answer options (A, B, C, D).
    3. Metadata including:
       - Topic
       - Subtopic
       - Cognitive Level
       - Expected Solution Time

    Use the following JSON format for each question:
    {
        "question": "<question stem>",
        "options": ["<option A>", "<option B>", "<option C>", "<option D>"],
        "metadata": {
            "topic": "<topic>",
            "subtopic": "<subtopic>",
            "grade_level": "9",
            "cognitive_level": "<Bloom's taxonomy level>",
            "expected_solution_time": "<time in seconds>"
        }
    }
    return only the JSON output with nothing else and ensure that the whole response is a proper json file that can be interpreted.
    """

    # Prepare the message with the extracted text
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Extract and format the MCQs from this text:\n{full_text}")
    ]

    # Get response from OpenAI
    response = chat_model(messages)
    response_content = response.content.strip()
    response_content = re.sub(r'```json\n', '', response_content)
    response_content = re.sub(r'\n```', '', response_content)
    #response_content = ['[',response_content,']']

    # Parse and display the formatted questions
    try:
        formatted_questions = json.loads(response_content)
        return formatted_questions
    except json.JSONDecodeError:
        print("Error: Unable to parse the JSON output from OpenAI.")
        print(f"Response Content: {response_content}")
        return []

# Run the processing function and display the results
processed_questions = process_text_with_openai(full_text, chat_model)

# Display the formatted questions in a structured format
print("\nFormatted Questions Extracted:\n")
print(json.dumps(processed_questions, indent=4))



Formatted Questions Extracted:

[
    {
        "question": "If x = sec y, where y \u2208]\u03c0/2, \u03c0[, then dx/dy = ...",
        "options": [
            "-x\u221a(x\u00b2-1)",
            "x\u221a(x\u00b2-1)",
            "-x\u221a(x\u00b2+1)",
            "x\u221a(x\u00b2+1)"
        ],
        "metadata": {
            "topic": "Calculus",
            "subtopic": "Differentiation",
            "grade_level": "9",
            "cognitive_level": "Application",
            "expected_solution_time": "60"
        }
    },
    {
        "question": "lim x\u21921 (e^x - e)/(x - 1) = ...",
        "options": [
            "e",
            "-e",
            "1",
            "-1"
        ],
        "metadata": {
            "topic": "Calculus",
            "subtopic": "Limits",
            "grade_level": "9",
            "cognitive_level": "Analysis",
            "expected_solution_time": "45"
        }
    },
    {
        "question": "If lim x\u21920 (ln(x + 1))^k/x = 4, then k = ..

In [146]:
# Function to generate and modify MCQs step-by-step using LangChain
def generate_mcq_step_by_step(prompt,json_input = None):
    # Initial system message and instructions for iterative generation
    system_prompt = """
You are an advanced AI assistant with the task of creating high-quality multiple-choice questions (MCQs). Your goal is to iteratively build the questions step-by-step until they are fully complete. Follow these steps:

1. Generate initial questions:
   - Create basic question stems.
   - Explore alternative question formulations.

2. Add answer options:
   - Provide 4 options (A, B, C, D) for each question.
   - Use diverse methods to derive and verify answers.
   - Critically assess for potential biases or flaws.

3. Review and improve:
   - Refine for clarity and precision.
   - Consider edge cases and exceptions.
   - Re-examine using a different perspective.

4. **Add Metadata**:
   - Add metadata to each question, including:
     - Topic
     - Subtopic
     - Cognitive level
     - Expected solution time

5. **Compile Final Questions**:
   - Compile all completed questions into the final output format with structured JSON.



### Output Format:
Ensure each response is in valid JSON format with real data. Use the following structure for each output(you can't respond with anything other than json valid format):

{
  "questions": [
    {
      "question": "<question stem>",
      "options": ["<option A>", "<option B>", "<option C>", "<option D>"],
      "correct_answer": "<correct option>",
      "explanation": "<explanation>",
      "metadata": {
        "topic": "<topic>",
        "subtopic": "<subtopic>",
        "grade_level": "<grade level>",
        "cognitive_level": "<cognitive level>",
        "expected_solution_time": "<time>"
      }
    }
  ]
}
 return only the JSON output with nothing else and ensure that the whole response is a proper json file that can be interpreted.
"""
    json_prompt = f"\nWhile using the following sample questions as a reference for each step: \n {json_input}"
    # Start the conversation with the system prompt
    messages = [SystemMessage(content=system_prompt + json_prompt) ,HumanMessage(content=prompt )]

    # Store the intermediate results in each step
    current_questions = []

    # Steps for the iterative process
    steps = [
        "Generate the basic question stems for the MCQs.",
        "Add answer options and distractors to each question.",
        "Review the questions and options for clarity and accuracy.",
        "Add metadata to each question.",
        "Compile the final structured MCQs."
    ]

    # Process each step one by one
    for step_index, step_instruction in enumerate(steps):
        # Send current questions and instructions for the next modification step
        if step_index == 0:
            # For the first step, we just use the initial prompt
            print(f"Step {step_index + 1}: {step_instruction}")
        else:
            # Include the current question structure for subsequent steps
            questions_message = f"Here is the current state of the questions:\n{json.dumps({'questions': current_questions}, indent=4)}"
            messages.append(HumanMessage(content=questions_message))
            print(f"\nStep {step_index + 1}: {step_instruction}\n")

        # Instruct the assistant to modify based on the current step
        messages.append(HumanMessage(content=step_instruction))

        # Get response from the assistant
        response = chat_model(messages)

        # Parse and display raw response for debugging
        response_content = response.content.strip()
        response_content = re.sub(r'```json\n', '', response_content)
        response_content = re.sub(r'\n```', '', response_content)
        print(f"Raw response at step {step_index + 1}:\n{response_content}\n{'=' * 80}")

        # Convert the response to JSON
        try:
            step_data = json.loads(response_content)
            if "questions" in step_data:
                current_questions = step_data["questions"]
            else:
                print(f"Step {step_index + 1} did not produce valid questions JSON.")
        except json.JSONDecodeError:
            print(f"Error decoding JSON at step {step_index + 1}: {response_content}")
            break

        # Add the assistant's response to the message history
        messages.append(AIMessage(content=response_content))

    # Final Output
    print("\nFinal MCQs Generated:\n")
    if current_questions:
        final_output = {"questions": current_questions}
        print(json.dumps(final_output, indent=4))
    else:
        print("No questions were generated.")

    return current_questions


In [148]:
sample_prompt = "Create 25 multiple-choice questions for the high school final calculus exam"

# Generate the MCQs using the step-by-step approach
generated_mcqs = generate_mcq_step_by_step(sample_prompt)

# Display the final MCQ generated in a readable format
if generated_mcqs:
    print("\nGenerated Questions:\n")
    for question in generated_mcqs:
        print(f"Question: {question['question']}")
        print(f"Options: {question['options']}")
        print(f"Correct Answer: {question['correct_answer']}")
        print(f"Explanation: {question['explanation']}")
        print(f"Metadata: {question['metadata']}\n")
else:
    print("No questions were generated.")


Step 1: Generate the basic question stems for the MCQs.
Raw response at step 1:
{
  "questions": [
    {
      "question": "What is the derivative of the function f(x) = x^2 + 3x + 5?",
      "options": [],
      "correct_answer": "",
      "explanation": "",
      "metadata": {
        "topic": "Calculus",
        "subtopic": "Derivatives",
        "grade_level": "High School",
        "cognitive_level": "Understanding",
        "expected_solution_time": "2 minutes"
      }
    },
    {
      "question": "Evaluate the integral of f(x) = 3x^2 from x = 0 to x = 2.",
      "options": [],
      "correct_answer": "",
      "explanation": "",
      "metadata": {
        "topic": "Calculus",
        "subtopic": "Integrals",
        "grade_level": "High School",
        "cognitive_level": "Application",
        "expected_solution_time": "3 minutes"
      }
    },
    {
      "question": "What is the limit of (3x^2 - 2x + 1) as x approaches 2?",
      "options": [],
      "correct_answer": "",

In [151]:
def generate_mcq_step_by_step(prompt, example_exam=None, course_outline=None):
    """
    This function generates high-quality multiple-choice questions (MCQs) step-by-step using an example exam and course outline as references.

    Parameters:
    - prompt: Initial user instruction for exam generation.
    - example_exam: A sample exam (full JSON structure) to be used as a one-shot learning guide.
    - course_outline: A structured outline of the course topics and subtopics to ensure comprehensive coverage.

    Returns:
    - A JSON object with the final set of generated MCQs.
    """
    # Initial system message and instructions for iterative generation
    system_prompt = """
You are an advanced AI assistant tasked with creating high-quality multiple-choice questions (MCQs). Your goal is to iteratively build the questions step-by-step until they are fully complete. Follow these steps:

1. Generate initial questions:
   - Create basic question stems based on the topics and subtopics from the provided course outline.
   - Use the structure and format of the provided example exam as a guide.

2. Add answer options:
   - Provide 4 options (A, B, C, D) for each question.
   - Use diverse methods to derive and verify answers.
   - Critically assess for potential biases or flaws.
   - Ensure the distractors follow patterns similar to those in the example exam.

3. Review and improve:
   - Refine for clarity and precision.
   - Consider edge cases and exceptions.
   - Re-examine using a different perspective.

4. **Add Metadata**:
   - Add metadata to each question, including:
     - Topic (from course outline)
     - Subtopic (from course outline)
     - Grade level
     - Cognitive level
     - Expected solution time

5. **Compile Final Questions**:
   - Compile all completed questions into the final output format with structured JSON.

### Output Format:
Ensure each response is in valid JSON format with real data. Use the following structure for each output(you can't respond with anything other than json valid format):

{
  "questions": [
    {
      "question": "<question stem>",
      "options": ["<option A>", "<option B>", "<option C>", "<option D>"],
      "correct_answer": "<correct option>",
      "explanation": "<explanation>",
      "metadata": {
        "topic": "<topic>",
        "subtopic": "<subtopic>",
        "grade_level": "<grade level>",
        "cognitive_level": "<cognitive level>",
        "expected_solution_time": "<time>"
      }
    }
  ]
}
 return only the JSON output with nothing else and ensure that the whole response is a proper json file that can be interpreted.
"""

    # Include example exam and course outline if provided
    exam_message = f"\nUse the following example exam as a reference for structure, format, and style:\n{json.dumps(example_exam, indent=4)}" if example_exam else ""
    outline_message = f"\nCreate questions based on the following course outline:\n{course_outline}" if course_outline else ""

    # Start the conversation with the system prompt, example exam, and course outline
    messages = [SystemMessage(content=system_prompt + exam_message + outline_message), HumanMessage(content=prompt)]

    # Store the intermediate results in each step
    current_questions = []

    # Steps for the iterative process
    steps = [
        "Generate the basic question stems for the MCQs.",
        "Add answer options and distractors to each question.",
        "Review the questions and options for clarity and accuracy.",
        "Add metadata to each question.",
        "Compile the final structured MCQs."
    ]

    # Process each step one by one
    for step_index, step_instruction in enumerate(steps):
        # Send current questions and instructions for the next modification step
        if step_index == 0:
            # For the first step, we just use the initial prompt
            print(f"Step {step_index + 1}: {step_instruction}")
        else:
            # Include the current question structure for subsequent steps
            questions_message = f"Here is the current state of the questions:\n{json.dumps({'questions': current_questions}, indent=4)}"
            messages.append(HumanMessage(content=questions_message))
            print(f"\nStep {step_index + 1}: {step_instruction}\n")

        # Instruct the assistant to modify based on the current step
        messages.append(HumanMessage(content=step_instruction))

        # Get response from the assistant
        response = chat_model(messages)

        # Parse and display raw response for debugging
        response_content = response.content.strip()
        response_content = re.sub(r'```json\n', '', response_content)
        response_content = re.sub(r'\n```', '', response_content)
        print(f"Raw response at step {step_index + 1}:\n{response_content}\n{'=' * 80}")

        # Convert the response to JSON
        try:
            step_data = json.loads(response_content)
            if "questions" in step_data:
                current_questions = step_data["questions"]
            else:
                print(f"Step {step_index + 1} did not produce valid questions JSON.")
        except json.JSONDecodeError:
            print(f"Error decoding JSON at step {step_index + 1}: {response_content}")
            break

        # Add the assistant's response to the message history
        messages.append(AIMessage(content=response_content))

    # Final Output
    print("\nFinal MCQs Generated:\n")
    if current_questions:
        final_output = {"questions": current_questions}
        print(json.dumps(final_output, indent=4))
    else:
        print("No questions were generated.")

    return current_questions


In [152]:
sample_prompt = ""

# Generate the MCQs using the step-by-step approach
generated_mcqs = generate_mcq_step_by_step(sample_prompt,str(json.dumps(processed_questions, indent=4)))

# Display the final MCQ generated in a readable format
if generated_mcqs:
    print("\nGenerated Questions:\n")
    for question in generated_mcqs:
        print(f"Question: {question['question']}")
        print(f"Options: {question['options']}")
        print(f"Correct Answer: {question['correct_answer']}")
        print(f"Explanation: {question['explanation']}")
        print(f"Metadata: {question['metadata']}\n")
else:
    print("No questions were generated.")


Step 1: Generate the basic question stems for the MCQs.
Raw response at step 1:
{
  "questions": [
    {
      "question": "What is the derivative of the function f(x) = x^2 + 3x + 5?",
      "options": [],
      "correct_answer": "",
      "explanation": "",
      "metadata": {
        "topic": "Calculus",
        "subtopic": "Differentiation",
        "grade_level": "12",
        "cognitive_level": "Application",
        "expected_solution_time": "60"
      }
    },
    {
      "question": "Evaluate the limit: lim x→0 (sin x)/x.",
      "options": [],
      "correct_answer": "",
      "explanation": "",
      "metadata": {
        "topic": "Calculus",
        "subtopic": "Limits",
        "grade_level": "12",
        "cognitive_level": "Analysis",
        "expected_solution_time": "45"
      }
    },
    {
      "question": "Find the integral of f(x) = 3x^2 with respect to x.",
      "options": [],
      "correct_answer": "",
      "explanation": "",
      "metadata": {
        "topi

### MORE IMPROVEMENT

In [153]:
def generate_response(prompt):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant with advanced reasoning capabilities. Your task is to provide detailed, step-by-step explanations of your thought process. For each step:

1. Provide a clear, concise title describing the current reasoning phase.
2. Elaborate on your thought process in the content section.
3. Decide whether to continue reasoning or provide a final answer.

Response Format:
Use JSON with keys: 'title', 'content', 'next_action' (values: 'continue' or 'final_answer')

Key Instructions:
- Employ at least 5 distinct reasoning steps.
- Acknowledge your limitations as an AI and explicitly state what you can and cannot do.
- Actively explore and evaluate alternative answers or approaches.
- Critically assess your own reasoning; identify potential flaws or biases.
- When re-examining, employ a fundamentally different approach or perspective.
- Utilize at least 3 diverse methods to derive or verify your answer.
- Incorporate relevant domain knowledge and best practices in your reasoning.
- Quantify certainty levels for each step and the final conclusion when applicable.
- Consider potential edge cases or exceptions to your reasoning.
- Provide clear justifications for eliminating alternative hypotheses.


Example of a valid JSON response:
```json
{
    "title": "Initial Problem Analysis",
    "content": "To approach this problem effectively, I'll first break down the given information into key components. This involves identifying...[detailed explanation]... By structuring the problem this way, we can systematically address each aspect.",
    "next_action": "continue"
}```
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant",
         "content": "Thank you! I will now think step by step following my instructions, starting at the beginning after decomposing the problem."}
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 300)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        messages.append({"role": "assistant", "content": json.dumps(step_data)})

        if step_data['next_action'] == 'final_answer':
            break

        step_count += 1

        # Yield after each step for Streamlit to update
        yield steps, None  # We're not yielding the total time until the end

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based on your reasoning above."})

    start_time = time.time()
    final_data = make_api_call(messages, 200, is_final_answer=True)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    steps.append(("Final Answer", final_data['content'], thinking_time))

    yield steps, total_thinking_time

In [171]:
def make_api_call(messages, max_tokens, is_final_answer=False):
    for attempt in range(3):
        try:
            response = client.chat.completions.create(
                model="gpt-4o",  # Using GPT-4, adjust as needed
                messages=messages,
                #max_tokens=max_tokens,
                temperature=0.5
            )
            return json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
            # If JSON parsing fails, return the raw content
            return {"title": "Parsing Error", "content": response.choices[0].message.content, "next_action": "final_answer"}
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error",
                            "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}",
                            "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying

def generate_response1(prompt):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant with advanced reasoning capabilities. Your task is to provide detailed, step-by-step explanations of your thought process. For each step:

1. Provide a clear, concise title describing the current reasoning phase.
2. Elaborate on your thought process in the content section.
3. Decide whether to continue reasoning or provide a final answer.

Response Format:
Use JSON with keys: 'title', 'content', 'next_action' (values: 'continue' or 'final_answer')

Key Instructions:
- Employ at least 5 distinct reasoning steps.
- Acknowledge your limitations as an AI and explicitly state what you can and cannot do.
- Actively explore and evaluate alternative answers or approaches.
- Critically assess your own reasoning; identify potential flaws or biases.
- When re-examining, employ a fundamentally different approach or perspective.
- Utilize at least 3 diverse methods to derive or verify your answer.
- Incorporate relevant domain knowledge and best practices in your reasoning.
- Quantify certainty levels for each step and the final conclusion when applicable.
- Consider potential edge cases or exceptions to your reasoning.
- Provide clear justifications for eliminating alternative hypotheses.
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant",
         "content": "Understood. I will now think through this step-by-step, following the given instructions and starting by decomposing the problem."}
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 1000)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        messages.append({"role": "assistant", "content": json.dumps(step_data)})

        if step_data['next_action'] == 'final_answer':
            break

        step_count += 1

        # Print each step
        print(f"\nStep {step_count}: {step_data['title']}")
        print(step_data['content'])
        print(f"Thinking time: {thinking_time:.2f} seconds")

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based on your reasoning above."})

    start_time = time.time()
    final_data = make_api_call(messages, 200, is_final_answer=True)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    steps.append(("Final Answer", final_data['content'], thinking_time))

    # Print final answer
    print("\nFinal Answer:")
    print(final_data['content'])
    print(f"Thinking time: {thinking_time:.2f} seconds")

    print(f"\nTotal thinking time: {total_thinking_time:.2f} seconds")

    return steps, total_thinking_time

print("OpenAI Reasoning Chains")
print("This is a prototype using OpenAI's GPT model to create reasoning chains for improved output accuracy.")
print("The accuracy has not been formally evaluated yet.")

# while True:
#     user_query = input("\nEnter your query (or 'quit' to exit): ")

#     if user_query.lower() == 'quit':
#         break

#     print("\nGenerating response...")

#     generate_response(user_query)



OpenAI Reasoning Chains
This is a prototype using OpenAI's GPT model to create reasoning chains for improved output accuracy.
The accuracy has not been formally evaluated yet.


In [159]:
generate_response("how many people in the world")[0][-1][1]


Step 2: Step 1: Understanding the Question
The question is asking for the current global population. This is a dynamic value that changes constantly due to births, deaths, and migration. As an AI, I don't have real-time data access, but I can provide the most recent estimate based on available data sources.
Thinking time: 3.24 seconds

Step 3: Step 2: Identifying Reliable Data Sources
For accurate and up-to-date population data, reliable sources include international organizations like the United Nations (UN), the World Bank, and the U.S. Census Bureau's International Data Base. These organizations regularly publish estimates and projections of the world population. However, as an AI, I can't access real-time data or updates from these sources. I can only provide information based on the data I was last trained on.
Thinking time: 4.92 seconds

Step 4: Step 3: Providing the Most Recent Estimate
The most recent data I was trained on, as of 2021, estimated the global population to be app

'Based on the most recent data I was trained on, the estimated global population as of 2021 is approximately 7.8 billion people. Please note that this is an estimate and the actual number may be slightly different due to the dynamic nature of population change. For the most accurate and up-to-date information, please check with reliable sources like the UN, the World Bank, or the U.S. Census Bureau.'

In [167]:
import json

def generate_response(prompt):
    # This function would call the desired model to generate a response.
    # Placeholder implementation: replace with your model-specific code or API call.
    # For example, this could be an API call or a LangChain model function call.
    return generate_response1(prompt)[0][-1][1]

def generate_question_stems(course_outline, example_exam=None):
    """
    Step 1: Generate basic question stems.
    This function takes a course outline and optional example exam as input and outputs basic question stems.
    """
    prompt = f"Generate 25 question stems based on the following course outline:\n{json.dumps(course_outline, indent=4)}"
    if example_exam:
        prompt += f"\nUse this example exam for style reference:\n{json.dumps(example_exam, indent=4)} and ensure that the response is an output json valid json format and follows exactly the same style as the reference exam"

    response_content = str(generate_response(prompt))
    response_content = response_content.strip()
    response_content = re.sub(r'```json\n', '', response_content)
    response_content = re.sub(r'\n```', '', response_content)
    # Parse the response and extract question stems (simulation)
    question_stems = [{"question": f"Sample Question Stem {i+1}"} for i in range(5)]  # Dummy stems for illustration
    return response_content

def generate_answer_options(question_stems, example_exam=None):
    """
    Step 2: Generate answer options for the provided question stems.
    """
    question_prompts = []
    for q in question_stems:
        # Prepare a specific prompt for each question
        prompt = f"Generate 4 answer options for the following question stem:\nQuestion: {q['question']}"
        if example_exam:
            prompt += f"\nUse this example exam for reference:\n{json.dumps(example_exam, indent=4)}"
        question_prompts.append(prompt)

    # Generate options for each question
    response_list = [generate_response(prompt) for prompt in question_prompts]

    # Example parsing logic to convert response into structured options (simulation)
    for idx, response in enumerate(response_list):
        question_stems[idx]["options"] = ["Option A", "Option B", "Option C", "Option D"]  # Dummy options
        question_stems[idx]["correct_answer"] = "Option A"  # Dummy correct answer

    return question_stems

def review_and_improve(questions):
    """
    Step 3: Review and improve the questions for clarity and quality.
    """
    prompt = f"Review the following questions for clarity and suggest improvements:\n{json.dumps(questions, indent=4)}"
    response = generate_response(prompt)

    # Example refinement logic (simulation)
    # This can include adjusting language, fixing errors, etc.
    for question in questions:
        question["question"] += " (Refined)"

    return questions

def add_metadata(questions, course_outline, grade_level="Grade 9"):
    """
    Step 4: Add metadata to each question based on the course outline and other specifications.
    """
    # Assign topic and subtopic from the course outline to each question
    for idx, question in enumerate(questions):
        topic, subtopic = list(course_outline.items())[idx % len(course_outline)]
        question["metadata"] = {
            "topic": topic,
            "subtopic": subtopic,
            "grade_level": grade_level,
            "cognitive_level": "Understanding",
            "expected_solution_time": "2 minutes"
        }

    return questions

def compile_final_exam(questions):
    """
    Step 5: Compile the questions into a structured final exam format.
    """
    final_exam = {"questions": questions}
    return final_exam

# Example usage of the step-by-step approach
course_outline = {
    "Algebra": ["Solving Equations", "Factoring"],
    "Geometry": ["Triangles", "Circles"],
    "Statistics": ["Mean and Median", "Standard Deviation"]
}

example_exam = {
    "questions": [
        {
            "question": "What is the sum of the angles in a triangle?",
            "options": ["90 degrees", "180 degrees", "270 degrees", "360 degrees"],
            "correct_answer": "180 degrees",
            "metadata": {
                "topic": "Geometry",
                "subtopic": "Triangles",
                "grade_level": "Grade 9",
                "cognitive_level": "Knowledge",
                "expected_solution_time": "1 minute"
            }
        }
    ]
}

# Step-by-step question generation
question_stems = generate_question_stems(course_outline, processed_questions)
questions_with_options = generate_answer_options(question_stems, example_exam)
refined_questions = review_and_improve(questions_with_options)
questions_with_metadata = add_metadata(refined_questions, course_outline)
final_exam = compile_final_exam(questions_with_metadata)

# Output the final exam
print(json.dumps(final_exam, indent=4))



Final Answer:
```json
{
    "questions": [
        {
            "question": "What is the solution to the equation 2x + 3 = 7?",
            "options": [
                "x = 1",
                "x = 2",
                "x = 3",
                "x = 4"
            ],
            "correct_answer": "x = 2",
            "metadata": {
                "topic": "Algebra",
                "subtopic": "Solving Equations",
                "grade_level": "Grade 9",
                "cognitive_level": "Application",
                "expected_solution_time": "2 minutes"
            }
        },
        {
            "question": "How do you solve the equation x^2 - 4 = 0?",
            "options": [
                "x = 2",
                "x = -2",
                "x = ±2",
                "x = 0"
            ],
            "correct_answer": "
Thinking time: 2.17 seconds

Total thinking time: 11.37 seconds


TypeError: string indices must be integers

In [188]:
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=openai_api_key)

def process_outline_with_openai(full_text, chat_model):
    """Processes the extracted text to identify and format MCQs using OpenAI."""
    # Define the system prompt
    system_prompt = """
    You are an advanced AI assistant that extracts and processes Course Outline from text.
    Please extract the following for each identified question:

    1. Main topics
    2. suptopics of the main topics
    3. Learning outcomes and brief content

    **output format**:
     topic:
        -subtopic:
            -learning outcomes:
    """

    # Prepare the message with the extracted text
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Extract and format the MCQs from this text:\n{full_text}")
    ]

    # Get response from OpenAI
    response = chat_model(messages)
    response_content = response.content.strip()
    response_content = re.sub(r'```json\n', '', response_content)
    response_content = re.sub(r'\n```', '', response_content)
    #response_content = ['[',response_content,']']

    return response_content

# Run the processing function and display the results
processed_outline = process_outline_with_openai(full_text, chat_model)

print(processed_outline)

1. Differentiation and its Applications
   - Differentiation of trigonometric functions:
     - Learning outcomes: To find derivatives of the inverse of trigonometric functions (secant, cosecant, cotangent).
   - Implicit and parametric differentiation:
     - Learning outcomes: To find derivatives of explicit, implicit, parametric functions and solve related problems.
   - Higher derivatives of the function:
     - Learning outcomes: To solve problems on higher order (second, third) derivatives of different functions.
   - Two equations of the tangent and normal to a curve:
     - Learning outcomes: To find the tangent and vertical line equations of a curve at a point as an application of differentiation.
   - Related time rates:
     - Learning outcomes: To find correlated time rates of a relation between variables, including physical, geometric, and real-life applications.

2. The calculus of Exponential and Logarithmic functions
   - The exponential function with the natural base a

In [190]:
import json

def generate_response(prompt):
    # This function would call the desired model to generate a response.
    # Placeholder implementation: replace with your model-specific code or API call.
    # For example, this could be an API call or a LangChain model function call.
    return generate_response1(prompt)[0][-1][1]

def generate_question_stems(course_outline, example_exam=None):
    """
    Step 1: Generate high-quality question stems based on the course outline and example exam.
    """
    prompt = f"""
Create challenging and high-quality 5 question stems based on the following course outline topics and subtopics:
{json.dumps(course_outline, indent=4)}.

- Ensure questions are clear, concise, and target critical concepts in each subtopic.
- Use the example exam as a style reference to match the difficulty and question type, if available.
- Include a variety of question types (definition, calculation, application, and reasoning) to cover different cognitive levels.

### Output format:
  output should be regular text without any formats
  -Question <question number >: <question stem>

"""

    if example_exam:
        prompt += f"\nExample exam for reference:\n{json.dumps(example_exam, indent=4)}"

    response = generate_response(prompt)
    response_content = response.strip()
    # response_content = re.sub(r'```json\n', '', response_content)
    # response_content = re.sub(r'\n```', '', response_content)
    # Parse the response and extract question stems
    # Simulate generating question stems based on the course outline
    #question_stems = [{"question": f"Sample Question Stem {i+1} on {list(course_outline.keys())[i % len(course_outline)]}"} for i in range(5)]
    return response_content

def generate_answer_options(question_stems, example_exam=None):
    """
    Step 2: Generate effective answer options and strong distractors for each question stem.
    """

    prompt = f"""
Create four answer options (A, B, C, D) for the following questions stems:
{question_stems}

**Requirements:**
- One correct answer and three high-quality distractors.
- Distractors should be plausible and designed to reveal common misconceptions or errors.
- Vary the difficulty of the options: at least one should be very close to the correct answer, and others should include common mistakes.
- Use the example exam as a reference for the complexity and style of options, if provided.

### Output format:
output should be regular text without any formats
  -Question <question number >: <question stem>
  -Option A: <option A>
  -Option B: <option B>
  -Option C: <option C>
  -Option D: <option D>
  -Correct Answer: <correct answer>
"""
    question_stems = generate_response(prompt)
    return question_stems

def review_and_improve(questions):
    """
    Step 3: Review and refine questions for clarity, quality, and appropriate difficulty.
    """
    prompt = f"""
Review the following questions for clarity, quality, and difficulty. Suggest improvements or refinements for each question:

{questions}

**Refinement Criteria:**
- Ensure that the question stem and options are free from ambiguity.
- Balance the difficulty: identify if any options are too obvious or too confusing.
- Improve the distractors if they do not adequately challenge the understanding of the topic.
- Adjust the language and format to ensure they align with a high-quality exam standard.

### Output format:
output should be regular text without any formats
  -Question <question number >: <question stem>
  -Option A: <option A>
  -Option B: <option B>
  -Option C: <option C>
  -Option D: <option D>
  -Correct Answer: <correct answer>
"""

    response = generate_response(prompt)

    # Example refinement logic (simulation)


    return response

def add_metadata(questions, course_outline, grade_level="Grade 9"):
    """
    Step 4: Add meaningful metadata for each question, reflecting cognitive complexity and expected solution time.
    """
    prompt = f"""
Analyze the following multiple-choice questions and extract detailed metadata for each one. Ensure the metadata captures the cognitive complexity, educational objectives, topic relevance, and expected solution time.

**Instructions:**
1. **Cognitive Level**: Determine the cognitive level of the question using Bloom’s Taxonomy (e.g., Knowledge, Comprehension, Application, Analysis, Synthesis, Evaluation). Base this on the type of thinking required to solve the question.
2. **Difficulty Level**: Classify the difficulty as one of the following:
   - **Easy**: Basic recall or simple calculations.
   - **Medium**: Requires reasoning or understanding of the concept.
   - **Hard**: Involves multi-step problem solving, advanced reasoning, or complex analysis.
3. **Topic and Subtopic**: Identify the specific topic and subtopic being tested (e.g., Topic: Algebra, Subtopic: Solving Quadratic Equations).
4. **Grade Level**: Specify the appropriate grade level for this question (e.g., Grade 7, Grade 9).
5. **Expected Solution Time**: Estimate the time required for a typical student to solve this question (e.g., “2 minutes”).
6. **Question Type**: Classify the type of question (e.g., Conceptual, Calculation, Application, Word Problem).
7. **Educational Objective**: Briefly describe the learning objective this question aims to assess (e.g., “Test understanding of the relationship between complementary angles.”).
8. **Misconception Identification**: If applicable, highlight any misconceptions that the distractors are designed to address (e.g., “Distractor A targets a common error in simplifying expressions with negative exponents.”).

**Input Questions:**
{questions}

**Output Format:**
output should be regular text without any formats
-Question <question number >: <question stem>
  -Option A: <option A>
  -Option B: <option B>
  -Option C: <option C>
  -Option D: <option D>
  -Correct Answer: <correct answer>
  -Metadata:
    -Cognitive Level: <cognitive level>
    -Difficulty Level: <difficulty level>
    -Topic: <topic>
    -Subtopic: <subtopic>
    -Grade Level: <grade level>
    -Expected Solution Time: <expected solution time>

"""
    questions = generate_response(prompt)
    return questions

def compile_final_exam(questions):
    """
    Step 5: Compile the refined questions into a final exam format with metadata.
    """
    final_exam = {"questions": questions}
    return final_exam

# Example usage of the step-by-step approach with enhanced prompts
course_outline = {
    "Algebra": ["Solving Equations", "Factoring"],
    "Geometry": ["Triangles", "Circles"],
    "Statistics": ["Mean and Median", "Standard Deviation"]
}

example_exam = {
    "questions": [
        {
            "question": "What is the sum of the angles in a triangle?",
            "options": ["90 degrees", "180 degrees", "270 degrees", "360 degrees"],
            "correct_answer": "180 degrees",
            "metadata": {
                "topic": "Geometry",
                "subtopic": "Triangles",
                "grade_level": "Grade 9",
                "cognitive_level": "Knowledge",
                "expected_solution_time": "1 minute"
            }
        }
    ]
}


#print(questions_with_metadata)
#final_exam = compile_final_exam(questions_with_metadata)

# Output the final exam
#print(json.dumps(final_exam, indent=4))


In [195]:
# Step-by-step question generation with enhanced prompts
question_stems = generate_question_stems(processed_outline, processed_questions)
#print(question_stems)
questions_with_options = generate_answer_options(question_stems, processed_questions)
#print(questions_with_options)
refined_questions = review_and_improve(questions_with_options)
#print(refined_questions)
questions_with_metadata = add_metadata(refined_questions, processed_outline)


Final Answer:
- Question 1: Given that y = sec⁻¹(x), where x > 1, find dy/dx in terms of x.

- Question 2: Consider the parametric equations x = t² + 1 and y = t³ - t. Find the second derivative d²y/dx² at t = 1.

- Question 3: Determine the equations of the tangent and normal lines to the curve y = x³ - 3x at the point where x = 1.

- Question 4: A spherical balloon is being inflated such that its volume increases at a rate of 100 cm³/s. Find the rate at which the radius of the balloon is increasing when the radius is 10 cm.

- Question 5: Evaluate the limit lim (x→0) (e^x - 1)/x and explain its significance in the context of derivatives of exponential functions.
Thinking time: 2.17 seconds

Total thinking time: 3.35 seconds

Final Answer:
- Question 1: Given that y = sec⁻¹(x), where x > 1, find dy/dx in terms of x.
  - Option A: 1 / (|x|√(x² - 1))
  - Option B: 1 / √(x² - 1)
  - Option C: x / (x² - 1)
  - Option D: 1 / (x² - 1)
  - Correct Answer: Option A

- Question 2: Consider th

# THE LAST MODEL




In [ ]:
import PyPDF2
import json
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import json

# Set up OpenAI API key (Make sure to securely input your API key in Colab)
openai_api_key = userdata.get('OPEN_AI_KEY')

chat_model = ChatOpenAI(model="gpt-4o", temperature=0.2, api_key=openai_api_key)

In [ ]:
def extract_text_from_pdf(pdf_path):
    """Extracts and returns the full text from a PDF file."""
    try:
        # Read the PDF file
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            full_text = ""
            for page in reader.pages:
                full_text += page.extract_text() + "\n"
        return full_text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""


In [ ]:
def process_exam_with_openai(full_text, chat_model):
    """Processes the extracted text to identify and format MCQs using OpenAI."""
    # Define the system prompt
    system_prompt = """
    You are an advanced AI assistant that extracts and processes multiple-choice questions (MCQs) from text.
    Please extract the following for each identified question:

    1. The question stem.
    2. Four answer options (A, B, C, D).
    3. Metadata including:
       - Topic
       - Subtopic
       - Cognitive Level
       - Expected Solution Time

    Use the following JSON format for each question:
    {
        "question": "<question stem>",
        "options": ["<option A>", "<option B>", "<option C>", "<option D>"],
        "metadata": {
            "topic": "<topic>",
            "subtopic": "<subtopic>",
            "grade_level": "9",
            "cognitive_level": "<Bloom's taxonomy level>",
            "expected_solution_time": "<time in seconds>"
        }
    }
    return only the JSON output with nothing else and ensure that the whole response is a proper json file that can be interpreted.
    """

    # Prepare the message with the extracted text
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Extract and format the MCQs from this text:\n{full_text}")
    ]

    # Get response from OpenAI
    response = chat_model(messages)
    response_content = response.content.strip()
    response_content = re.sub(r'```json\n', '', response_content)
    response_content = re.sub(r'\n```', '', response_content)
    #response_content = ['[',response_content,']']

    # Parse and display the formatted questions
    try:
        formatted_questions = json.loads(response_content)
        return formatted_questions
    except json.JSONDecodeError:
        print("Error: Unable to parse the JSON output from OpenAI.")
        print(f"Response Content: {response_content}")
        return []



In [ ]:
def process_outline_with_openai(full_text, chat_model):
    """Processes the extracted text to identify and format MCQs using OpenAI."""
    # Define the system prompt
    system_prompt = """
    You are an advanced AI assistant that extracts and processes Course Outline from text.
    Please extract the following for each identified question:

    1. Main topics
    2. suptopics of the main topics
    3. Learning outcomes and brief content

    **output format**:
     topic:
        -subtopic:
            -learning outcomes:
    """

    # Prepare the message with the extracted text
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Extract and format the MCQs from this text:\n{full_text}")
    ]

    # Get response from OpenAI
    response = chat_model(messages)
    response_content = response.content.strip()
    response_content = re.sub(r'```json\n', '', response_content)
    response_content = re.sub(r'\n```', '', response_content)
    #response_content = ['[',response_content,']']

    return response_content

# Run the processing function and display the results


In [194]:
outline_path = '/content/differential-matter_e.pdf'
exam_path = '/content/calculus-english-session1-2021-prac.pdf'

exam_text = extract_text_from_pdf(exam_path)
outline_text = extract_text_from_pdf(outline_path)

In [ ]:
# Run the processing function and display the results
processed_questions = process_text_with_openai(full_text, chat_model)

print("\nFormatted Questions Extracted:\n")
processed_questions = json.dumps(processed_questions, indent=4)
print(processed_questions)


processed_outline = process_outline_with_openai(full_text, chat_model)
print("\nFormatted Outline Extracted:\n")
print(processed_outline)

In [ ]:
def make_api_call(messages, max_tokens, is_final_answer=False):
    for attempt in range(3):
        try:
            response = client.chat.completions.create(
                model="gpt-4o",  # Using GPT-4, adjust as needed
                messages=messages,
                #max_tokens=max_tokens,
                temperature=0.5
            )
            return json.loads(response.choices[0].message.content)
        except json.JSONDecodeError:
            # If JSON parsing fails, return the raw content
            return {"title": "Parsing Error", "content": response.choices[0].message.content, "next_action": "final_answer"}
        except Exception as e:
            if attempt == 2:
                if is_final_answer:
                    return {"title": "Error",
                            "content": f"Failed to generate final answer after 3 attempts. Error: {str(e)}"}
                else:
                    return {"title": "Error", "content": f"Failed to generate step after 3 attempts. Error: {str(e)}",
                            "next_action": "final_answer"}
            time.sleep(1)  # Wait for 1 second before retrying

def generate_response1(prompt):
    messages = [
        {"role": "system", "content": """You are an expert AI assistant with advanced reasoning capabilities. Your task is to provide detailed, step-by-step explanations of your thought process. For each step:

1. Provide a clear, concise title describing the current reasoning phase.
2. Elaborate on your thought process in the content section.
3. Decide whether to continue reasoning or provide a final answer.

Response Format:
Use JSON with keys: 'title', 'content', 'next_action' (values: 'continue' or 'final_answer')

Key Instructions:
- Employ at least 5 distinct reasoning steps.
- Acknowledge your limitations as an AI and explicitly state what you can and cannot do.
- Actively explore and evaluate alternative answers or approaches.
- Critically assess your own reasoning; identify potential flaws or biases.
- When re-examining, employ a fundamentally different approach or perspective.
- Utilize at least 3 diverse methods to derive or verify your answer.
- Incorporate relevant domain knowledge and best practices in your reasoning.
- Quantify certainty levels for each step and the final conclusion when applicable.
- Consider potential edge cases or exceptions to your reasoning.
- Provide clear justifications for eliminating alternative hypotheses.
"""},
        {"role": "user", "content": prompt},
        {"role": "assistant",
         "content": "Understood. I will now think through this step-by-step, following the given instructions and starting by decomposing the problem."}
    ]

    steps = []
    step_count = 1
    total_thinking_time = 0

    while True:
        start_time = time.time()
        step_data = make_api_call(messages, 1000)
        end_time = time.time()
        thinking_time = end_time - start_time
        total_thinking_time += thinking_time

        steps.append((f"Step {step_count}: {step_data['title']}", step_data['content'], thinking_time))

        messages.append({"role": "assistant", "content": json.dumps(step_data)})

        if step_data['next_action'] == 'final_answer':
            break

        step_count += 1

        # Print each step
        print(f"\nStep {step_count}: {step_data['title']}")
        print(step_data['content'])
        print(f"Thinking time: {thinking_time:.2f} seconds")

    # Generate final answer
    messages.append({"role": "user", "content": "Please provide the final answer based on your reasoning above."})

    start_time = time.time()
    final_data = make_api_call(messages, 200, is_final_answer=True)
    end_time = time.time()
    thinking_time = end_time - start_time
    total_thinking_time += thinking_time

    steps.append(("Final Answer", final_data['content'], thinking_time))

    # Print final answer
    print("\nFinal Answer:")
    print(final_data['content'])
    print(f"Thinking time: {thinking_time:.2f} seconds")

    print(f"\nTotal thinking time: {total_thinking_time:.2f} seconds")

    return steps, total_thinking_time

In [ ]:
def generate_response(prompt):
    # This function would call the desired model to generate a response.
    # Placeholder implementation: replace with your model-specific code or API call.
    # For example, this could be an API call or a LangChain model function call.
    return generate_response1(prompt)[0][-1][1]

def generate_question_stems(course_outline, example_exam=None):
    """
    Step 1: Generate high-quality question stems based on the course outline and example exam.
    """
    prompt = f"""
Create challenging and high-quality 5 question stems based on the following course outline topics and subtopics:
{json.dumps(course_outline, indent=4)}.

- Ensure questions are clear, concise, and target critical concepts in each subtopic.
- Use the example exam as a style reference to match the difficulty and question type, if available.
- Include a variety of question types (definition, calculation, application, and reasoning) to cover different cognitive levels.

### Output format:
  output should be regular text without any formats
  -Question <question number >: <question stem>

"""

    if example_exam:
        prompt += f"\nExample exam for reference:\n{json.dumps(example_exam, indent=4)}"

    response = generate_response(prompt)
    response_content = response.strip()
    # response_content = re.sub(r'```json\n', '', response_content)
    # response_content = re.sub(r'\n```', '', response_content)
    # Parse the response and extract question stems
    # Simulate generating question stems based on the course outline
    #question_stems = [{"question": f"Sample Question Stem {i+1} on {list(course_outline.keys())[i % len(course_outline)]}"} for i in range(5)]
    return response_content

def generate_answer_options(question_stems, example_exam=None):
    """
    Step 2: Generate effective answer options and strong distractors for each question stem.
    """

    prompt = f"""
Create four answer options (A, B, C, D) for the following questions stems:
{question_stems}

**Requirements:**
- One correct answer and three high-quality distractors.
- Distractors should be plausible and designed to reveal common misconceptions or errors.
- Vary the difficulty of the options: at least one should be very close to the correct answer, and others should include common mistakes.
- Use the example exam as a reference for the complexity and style of options, if provided.

### Output format:
output should be regular text without any formats
  -Question <question number >: <question stem>
  -Option A: <option A>
  -Option B: <option B>
  -Option C: <option C>
  -Option D: <option D>
  -Correct Answer: <correct answer>
"""
    question_stems = generate_response(prompt)
    return question_stems

def review_and_improve(questions):
    """
    Step 3: Review and refine questions for clarity, quality, and appropriate difficulty.
    """
    prompt = f"""
Review the following questions for clarity, quality, and difficulty. Suggest improvements or refinements for each question:

{questions}

**Refinement Criteria:**
- Ensure that the question stem and options are free from ambiguity.
- Balance the difficulty: identify if any options are too obvious or too confusing.
- Improve the distractors if they do not adequately challenge the understanding of the topic.
- Adjust the language and format to ensure they align with a high-quality exam standard.

### Output format:
output should be regular text without any formats
  -Question <question number >: <question stem>
  -Option A: <option A>
  -Option B: <option B>
  -Option C: <option C>
  -Option D: <option D>
  -Correct Answer: <correct answer>
"""

    response = generate_response(prompt)

    # Example refinement logic (simulation)


    return response

def add_metadata(questions, course_outline, grade_level="Grade 9"):
    """
    Step 4: Add meaningful metadata for each question, reflecting cognitive complexity and expected solution time.
    """
    prompt = f"""
Analyze the following multiple-choice questions and extract detailed metadata for each one. Ensure the metadata captures the cognitive complexity, educational objectives, topic relevance, and expected solution time.

**Instructions:**
1. **Cognitive Level**: Determine the cognitive level of the question using Bloom’s Taxonomy (e.g., Knowledge, Comprehension, Application, Analysis, Synthesis, Evaluation). Base this on the type of thinking required to solve the question.
2. **Difficulty Level**: Classify the difficulty as one of the following:
   - **Easy**: Basic recall or simple calculations.
   - **Medium**: Requires reasoning or understanding of the concept.
   - **Hard**: Involves multi-step problem solving, advanced reasoning, or complex analysis.
3. **Topic and Subtopic**: Identify the specific topic and subtopic being tested (e.g., Topic: Algebra, Subtopic: Solving Quadratic Equations).
4. **Grade Level**: Specify the appropriate grade level for this question (e.g., Grade 7, Grade 9).
5. **Expected Solution Time**: Estimate the time required for a typical student to solve this question (e.g., “2 minutes”).
6. **Question Type**: Classify the type of question (e.g., Conceptual, Calculation, Application, Word Problem).
7. **Educational Objective**: Briefly describe the learning objective this question aims to assess (e.g., “Test understanding of the relationship between complementary angles.”).
8. **Misconception Identification**: If applicable, highlight any misconceptions that the distractors are designed to address (e.g., “Distractor A targets a common error in simplifying expressions with negative exponents.”).

**Input Questions:**
{questions}

**Output Format:**
output should be regular text without any formats
-Question <question number >: <question stem>
  -Option A: <option A>
  -Option B: <option B>
  -Option C: <option C>
  -Option D: <option D>
  -Correct Answer: <correct answer>
  -Metadata:
    -Cognitive Level: <cognitive level>
    -Difficulty Level: <difficulty level>
    -Topic: <topic>
    -Subtopic: <subtopic>
    -Grade Level: <grade level>
    -Expected Solution Time: <expected solution time>

"""
    questions = generate_response(prompt)
    return questions


